In [ ]:
from functools import partial 

from sepal_ui import sepalwidgets as sw
from sepal_ui import mapping as sm
from scripts import messages as ms
from scripts import planet
import parameters as pm
import ipyvuetify as v

In [ ]:
class Order_io:
    
    def __init__(self):
        self.order_index = None
        self.orders = None
        self.session = None
        self.quads = None
        
po_order_io = Order_io()

In [ ]:
id_ = 'result_widget'

# create a button line 
down_theorical_grid = sw.DownloadBtn('download theorical grid')
down_grid = sw.DownloadBtn('download planet grid')
btn_line = v.Layout(Row=True, children=[down_theorical_grid, down_grid])

# map to display the grid 
res_map = sm.SepalMap()

res_tile = sw.Tile(
    id_,
    "Download results",
    inputs = [btn_line, res_map]
)

In [ ]:
id_ = 'download_widget'

#check input 
viz_output = sw.Alert()

viz_txt = sw.Markdown(ms.VIZ_TXT) 
viz_btn = sw.Btn(ms.VIZ_BTN)

viz_tile = sw.Tile(
    id_,
    "Check inputs",
    inputs = [viz_txt],
    output = viz_output,
    btn = viz_btn
)

In [ ]:
#check input 
order_output = sw.Alert()

order_txt = sw.Markdown(ms.ORDER_TXT) 
order_btn = sw.Btn(ms.ORDER_BTN, disabled=True)

order_tile = sw.Tile(
    id_,
    "Order Mosaic",
    inputs = [order_txt],
    output = order_output,
    btn = order_btn
)

In [ ]:
#check input 
down_output = sw.Alert()

down_txt = sw.Markdown(ms.DOWN_TXT) 
down_orders = v.Select(items=[], label=ms.SELECT_ORDER, v_model=None)
down_btn = sw.Btn(ms.DOWN_BTN, disabled=True)

down_output.bind(down_orders, po_order_io, 'order_index')

down_tile = sw.Tile(
    id_,
    "Download Mosaic",
    inputs = [down_txt, down_orders],  
    output = down_output,
    btn = down_btn
)

In [ ]:
def viz_data(widget, event, data, output):

    widget.toggle_loading()

    # check input 
    if not output.check_input(po_aoi_io.get_aoi_ee(), ms.NO_AOI): return widget.toggle_loading()
    
    try:
        # draw the grid on the map 
        grid_path = planet.get_theorical_grid(po_aoi_io, res_map, output)
        down_theorical_grid.set_url(str(grid_path))
    
        # write msg
        msg = planet.get_sum_up(po_aoi_io)
        output.add_msg(msg, 'warning')

        # release the second btn 
        order_btn.disabled = False
    except Exception as e:
        output.add_live_msg(str(e), 'error')
        
    widget.toggle_loading()

    return 

viz_btn.on_event('click', partial(
    viz_data,
    output = viz_output
))

In [ ]:
def order_mosaic(widget, event, data, output):

    widget.toggle_loading()

    # check input 
    if not output.check_input(po_aoi_io.get_aoi_ee(), ms.NO_AOI): return widget.toggle_loading()
 
    try:
        # get the orders
        orders, session = planet.get_orders(pm.PLANET_API_KEY, pm.BASEMAP_URL, output)
        po_order_io.orders = orders
        po_order_io.session = session
        
        # construct a order dict with only the name and the index
        #orders = {order['name']: i for i, order in enumerate(orders)}
        down_orders.items = [{'text': order['name'], 'value': i} for i, order in enumerate(orders)]
        down_btn.disabled = False
        
        output.add_live_msg('Orders list updated', 'success')
    
    except Exception as e: 
        output.add_live_msg(str(e), 'error') 

    widget.toggle_loading()

    return

order_btn.on_event('click', partial(
    order_mosaic,
    output = order_output
))

In [ ]:
toto = None

def down_mosaic(widget, event, data, output):

    widget.toggle_loading()

    # check input 
    if not output.check_input(po_aoi_io.get_aoi_ee(), ms.NO_AOI): return widget.toggle_loading()
    if not output.check_input(po_order_io.order_index, ms.NO_ORDER): return widget.toggle_loading()
 
    try:
        mosaic_path, grid_path = planet.run_download(pm.PLANET_API_KEY, pm.BASEMAP_URL, po_aoi_io, po_order_io, res_map, output)
        down_grid.set_url(str(grid_path))
        
    except Exception as e: 
        output.add_live_msg(str(e), 'error') 

    widget.toggle_loading()

    return

down_btn.on_event('click', partial(
    down_mosaic,
    output = down_output
))

In [ ]:
viz_tile

In [ ]:
order_tile

In [ ]:
down_tile

In [ ]:
res_tile

In [3]:
from pathlib import Path

tutu_dir = Path('~/toto/tutu').expanduser()
tutu_dir.name

'tutu'